# Creating a model and training it for generating and clustering music

In [1]:
#TODO: Create models: VAE, WAE, U-NET
#Save model weights
#Evaluate in this file?

In [131]:
#libraries
import deeplay as dl
import deeptrack as dt
import torch
import os
from PIL import Image, ImageOps
import torchvision.transforms as T
import matplotlib.pyplot as plt
import numpy as np
from skimage.color import rgb2gray

In [ ]:
#NOTE: Class for converting to graysscale, I dont think it is necessary anymore since all train images are grayscale from beginning now
class LoadImageGrayscale(dt.LoadImage):
    def get(self, *args, **kwargs):
        image = super().get(*args, **kwargs)
        if image.ndim == 3 and image.shape[-1] == 3:
            image = rgb2gray(image)
            image = np.expand_dims(image, axis=-1)  # Add channel dimension
        return image

In [110]:
#Preprocess data
data_dir = os.path.expanduser("./data/trainImages")

# Load image files using ImageFolder
trainFiles = dt.sources.ImageFolder(root=data_dir)

print(f"Number of train images: {len(trainFiles)}")
#print(f"Number of test images: {len(test_files)}")

Number of train images: 8246


In [133]:
#Create image pipeline
image_pip = (LoadImageGrayscale(trainFiles.path) >> dt.NormalizeMinMax()
             >> dt.MoveAxis(2, 0) >> dt.pytorch.ToTensor(dtype=torch.float))

In [134]:
img_tensor = image_pip(trainFiles.path[0])
print(img_tensor.shape)

torch.Size([1, 480, 640])


In [135]:
#VAE
vae = dl.VariationalAutoEncoder(input_size=(480,640),
    latent_dim=2, channels=[32, 64],
    reconstruction_loss=torch.nn.BCELoss(reduction="sum"), beta=1,
).create()

#print(vae)

In [16]:
#WAE
wae = dl.WassersteinAutoEncoder(
    channels=[32, 64, 128], latent_dim=20,
    reconstruction_loss=torch.nn.MSELoss(reduction="mean"),
).create()

#print(wae)

In [136]:
train_dataset = dt.pytorch.Dataset(image_pip & image_pip, inputs=trainFiles)
train_loader = dl.DataLoader(train_dataset, batch_size=64, shuffle=True)

In [137]:
vae_trainer = dl.Trainer(max_epochs=2, accelerator="auto")
vae_trainer.fit(vae, train_loader)

c:\Users\Lukas\AppData\Local\Programs\Python\Python312\Lib\site-packages\lightning\pytorch\trainer\configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.

  | Name                | Type                   | Params | Mode 
-----------------------------------------------------------------------
0 | encoder             | ConvolutionalEncoder2d | 55.7 K | train
1 | fc_mu               | Linear                 | 2.5 M  | train
2 | fc_var              | Linear                 | 2.5 M  | train
3 | fc_dec              | Linear                 | 3.7 M  | train
4 | decoder             | ConvolutionalDecoder2d | 76.3 K | train
5 | reconstruction_loss | BCELoss                | 0      | train
6 | train_metrics       | MetricCollection       | 0      | train
7 | val_metrics         | MetricCollection       | 0      | train
8 | test_metrics        | MetricCollection       | 0      | train
9 | optimizer           | Adam                   | 0    

Training: |          | 0/? [00:00<?, ?it/s]

NameError: name 'exit' is not defined